In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path
from tqdm import tqdm
# --- CONFIGURATION ---
canvas_width = 1280
canvas_height = 720
selective_mode = False  # Set to False to run for all participants

project_root = Path.cwd()
fixation_dir = project_root / "Fixation_Results"
output_folder = project_root / "Headlamp_Fixation_Analysis"
output_folder.mkdir(exist_ok=True)

output_pdf = output_folder / ("Headlamps_Fixations_Selective.pdf" if selective_mode else "Headlamps_Fixations_All.pdf")
output_summary_csv = output_folder / ("Illuminated_area_selective.csv" if selective_mode else "Illuminated_area_all.csv")
selective_participants = ["Charan", "Karim"]  # Modify as needed

In [ ]:
# --- Define Fixed Pixel Rectangle for Headlamp Illumination Area ---
x_min = 550.407
x_max = 750.451
y_top = 370.744
y_bottom = 470.762  # Note: lower part of canvas (in image terms)

# --- Helper Function ---
def extract_participant_name(filename):
    parts = filename.split("_")[0].strip().lower()
    return parts.capitalize()

# --- Load Nighttime Fixations ---
night_files = list(fixation_dir.glob("*_Nighttime_Fixations.csv"))
night_fixations = {extract_participant_name(f.stem): f for f in night_files}
participants = sorted(night_fixations.keys())

# --- Analysis ---
summary = []

with PdfPages(output_pdf) as pdf:
    for name in tqdm(selective_participants if selective_mode else participants, desc="Processing Participants"):
        if name not in night_fixations:
            print(f"Skipping {name}: No fixation data found.")
            continue

        df_night = pd.read_csv(night_fixations[name])

        # Identify fixations within illuminated area
        in_light_mask = (
            (df_night['fixation_x'] >= x_min) & (df_night['fixation_x'] <= x_max) &
            (df_night['fixation_y'] >= y_top) & (df_night['fixation_y'] <= y_bottom)
        )

        fixations_in_light = in_light_mask.sum()
        total_fixations = len(df_night)
        percent_in_light = (fixations_in_light / total_fixations) * 100 if total_fixations > 0 else 0

        summary.append({
            "Participant": name,
            "Total_Fixations": total_fixations,
            "Fixations_In_Light": fixations_in_light,
            "Percent_In_Light": percent_in_light
        })

        # ---- Plotting ----
        fig, ax = plt.subplots(figsize=(8, 6))
        fig.suptitle(f"{name} – Nighttime Fixations in Headlamp Area", fontsize=16)

        # Plot all fixations (red, behind)
        ax.scatter(df_night['fixation_x'], df_night['fixation_y'], c='red', s=10, label='All Fixations', alpha=0.5, zorder=1)

        # Plot fixations in illuminated area (green, in front)
        ax.scatter(
            df_night.loc[in_light_mask, 'fixation_x'],
            df_night.loc[in_light_mask, 'fixation_y'],
            c='limegreen', s=12, label='Fixations in Light', zorder=2
        )

        # Draw illuminated area rectangle
        rect = Rectangle(
            (x_min, y_top),
            x_max - x_min,
            y_bottom - y_top,
            linewidth=2,
            edgecolor='cyan',
            facecolor='none',
            label='Illuminated Area',
            zorder=3
        )
        ax.add_patch(rect)

        # Canvas formatting
        ax.set_xlim(0, canvas_width)
        ax.set_ylim(canvas_height, 0)  # Invert Y-axis (top-down view)
        ax.set_xlabel("Fixation X (px)")
        ax.set_ylabel("Fixation Y (px)")
        ax.legend(loc='upper right')
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        pdf.savefig(fig, dpi=300)
        plt.close(fig)

# --- Save Summary ---
summary_df = pd.DataFrame(summary)
summary_df.to_csv(output_summary_csv, index=False)

print(f"PDF saved to: {output_pdf}")
print(f"Summary CSV saved to: {output_summary_csv}")


In [ ]:
# ---- Bar Plot for Headlamp Fixations (Anonymized X-Labels) ----
import pandas as pd
import matplotlib.pyplot as plt

# Load the summary CSV
summary_df = pd.read_csv(output_summary_csv)
summary_df = summary_df.sort_values("Participant").reset_index(drop=True)

# Create bar plot
plt.figure(figsize=(10, 6))
bars = plt.bar(summary_df["Participant"], summary_df["Percent_In_Light"], color='skyblue', edgecolor='black')

# Replace participant names with "Participant 1", ..., "Participant N"
anonymized_labels = [f"Participant {i+1}" for i in range(len(summary_df))]
plt.xticks(ticks=range(len(summary_df)), labels=anonymized_labels, rotation=45, ha='right')

# Axis labels and formatting
plt.ylabel("Percent Fixations in Light")
plt.xlabel("Participant")
plt.ylim(0, 100)
plt.title("Fixation Percentage within Headlamp Illuminated Area (Nighttime)")
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()

# Save and close
plot_path = output_folder / "Fixation_Percentages_Headlamp_Nighttime_Anonymized.png"
plt.savefig(plot_path, dpi=300)
plt.close()

print(f"Headlamp fixation bar plot with anonymized labels saved to: {plot_path}")
